In [1]:
from pyspark.sql import SparkSession

# Crear SparkSession
key = "<API_KEY"

spark = SparkSession.builder \
    .appName("AzureDataLake") \
    .config("spark.hadoop.fs.azure.account.auth.type.corpusdata.dfs.core.windows.net", "SharedKey") \
    .config("spark.hadoop.fs.azure", "org.apache.hadoop.fs.azure.NativeAzureFileSystem") \
    .config("spark.hadoop.fs.azure.account.key.corpusdata.dfs.core.windows.net", key) \
    .config("spark.executor.memory", "8g") \
    .config("spark.driver.memory", "4g") \
    .config("spark.jars", "/app/jars/hadoop-azure-3.3.0.jar,/app/jars/hadoop-azure-datalake-3.3.0.jar,/app/jars/azure-storage-file-datalake-12.14.1.jar") \
    .getOrCreate()

/usr/local/lib/python3.10/site-packages/pyspark/bin/load-spark-env.sh: line 68: ps: command not found
25/04/09 19:12:17 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [2]:
# Definir el path del contenedor en Azure Data Lake
storage_account_name = "corpusdata"
container_name = "raw"
prefix = "year=2025/language=es"
path = f"abfss://{container_name}@{storage_account_name}.dfs.core.windows.net/{prefix}"

# Leer los archivos desde Azure Data Lake
df = spark.read.text(path)

# Mostrar algunas filas
df.show(5, truncate=False)

+-----------------------------------------------------------------------+
|value                                                                  |
+-----------------------------------------------------------------------+
|                                                                       |
|  En esta edición se han mantenido las convenciones ortográficas del   |
|original, incluyendo las variadas normas de acentuación presentes en el|
|                    texto. (nota del transcriptor)                     |
|                                                                       |
+-----------------------------------------------------------------------+
only showing top 5 rows



In [3]:
from pyspark.sql.functions import regexp_extract, regexp_replace
# Leer los archivos de texto completos con su ruta
rdd = spark.sparkContext.wholeTextFiles(path)

# Convertir a DataFrame
df_txt = rdd.toDF(["path", "content"])

# Extraer el nombre del archivo de la ruta (por ejemplo: don_quijote.txt)
from pyspark.sql.functions import regexp_extract, input_file_name

df_txt = df_txt.withColumn("nombre_archivo", regexp_extract("path", r"([^/]+\.txt)$", 1))
df_txt = df_txt.withColumn("nombre_archivo", regexp_replace("nombre_archivo", r"_text\.txt$", ""))

25/04/09 19:12:28 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 1 (TID 1): Attempting to kill Python Worker
                                                                                

In [4]:
import pandas as pd

# Leer CSV con pandas
url = "https://raw.githubusercontent.com/NelbaBarreto/programacion-ciencias-datos/refs/heads/main/data/metadata.csv"
df_pandas = pd.read_csv(url)

# Convertir a DataFrame de Spark
df_meta = spark.createDataFrame(df_pandas)

# Filtrar solo los registros donde language sea igual a 'es'
df_filtrado = df_meta.filter(df_meta.language == "['es']")

df_filtrado.show(5, truncate=False)

+-------+-----------------------------------------------------------------------------------------------------------------------+--------------+-----------------+-----------------+--------+---------+-------------------------------------------------------------+-----+
|id     |title                                                                                                                  |author        |authoryearofbirth|authoryearofdeath|language|downloads|subjects                                                     |type |
+-------+-----------------------------------------------------------------------------------------------------------------------+--------------+-----------------+-----------------+--------+---------+-------------------------------------------------------------+-----+
|PG10228|El Presidente Díaz al Señor Edison                                                                                     |Díaz, Porfirio|1830.0           |1915.0           |['es']  |38     

In [5]:
df_txt.show()

25/04/09 19:12:40 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 3 (TID 3): Attempting to kill Python Worker
                                                                                

+--------------------+--------------------+--------------+
|                path|             content|nombre_archivo|
+--------------------+--------------------+--------------+
|abfss://raw@corpu...|\n[Nota del Trans...|       PG10293|
|abfss://raw@corpu...|\nROMANCE DE LOBO...|       PG10506|
|abfss://raw@corpu...|\nLegends, Tales ...|       PG10814|
|abfss://raw@corpu...|\nMANFREDO, DRAMA...|       PG10821|
|abfss://raw@corpu...|\n    EL PARAÍSO ...|       PG10822|
|abfss://raw@corpu...|\n[Bold text is m...|       PG10825|
|abfss://raw@corpu...|\n      [Ilustrac...|       PG10909|
|abfss://raw@corpu...|\nCredits: John H...|       PG11047|
|abfss://raw@corpu...|\nLA FONTANA DE O...|       PG11070|
|abfss://raw@corpu...|\n[Nota del Trans...|       PG11071|
|abfss://raw@corpu...|\n[Nota del trans...|       PG11081|
|abfss://raw@corpu...|\nDIARIO DE LA NA...|       PG11302|
|abfss://raw@corpu...|\n             LA...|       PG11529|
|abfss://raw@corpu...|\n     LA MONTAÑA...|       PG1159

In [6]:
df_meta = df_filtrado.withColumnRenamed("id", "nombre_archivo")

df_final = df_txt.join(df_meta, on="nombre_archivo", how="left")

In [7]:
df_final.show()

25/04/09 19:12:48 WARN PythonRunner: Detected deadlock while completing task 0.0 in stage 4 (TID 4): Attempting to kill Python Worker
25/04/09 19:12:48 WARN PythonRunner: Detected deadlock while completing task 1.0 in stage 4 (TID 5): Attempting to kill Python Worker
                                                                                

+--------------+--------------------+--------------------+--------------------+--------------------+-----------------+-----------------+--------+---------+--------------------+----+
|nombre_archivo|                path|             content|               title|              author|authoryearofbirth|authoryearofdeath|language|downloads|            subjects|type|
+--------------+--------------------+--------------------+--------------------+--------------------+-----------------+-----------------+--------+---------+--------------------+----+
|       PG11598|abfss://raw@corpu...|\n     LA MONTAÑA...|          La Montaña|      Reclus, Elisée|           1830.0|           1905.0|  ['es']|       92|       {'Mountains'}|Text|
|       PG10814|abfss://raw@corpu...|\nLegends, Tales ...|                NULL|                NULL|             NULL|             NULL|    NULL|     NULL|                NULL|NULL|
|       PG10821|abfss://raw@corpu...|\nMANFREDO, DRAMA...|Manfredo: Drama e...|Byron, Geor

In [8]:
df_final.columns

['nombre_archivo',
 'path',
 'content',
 'title',
 'author',
 'authoryearofbirth',
 'authoryearofdeath',
 'language',
 'downloads',
 'subjects',
 'type']

In [9]:
df_final = df_final.drop('path', 'type', 'downloads', 'language')

In [10]:
from pyspark.ml.feature import Tokenizer, StopWordsRemover
from pyspark.sql.functions import expr
from pyspark.ml.feature import RegexTokenizer

# Tokenizar el texto
regex_tokenizer = RegexTokenizer(
    inputCol="content",
    outputCol="tokens",
    pattern="[^\p{L}]+",  # separa por todo lo que no sea letra (unicode)
    minTokenLength=2,
    gaps=True,  # pattern define lo que se usa como separador
    toLowercase=True  # podés poner esto en lugar de transform luego
)

# Tokenizar el texto
df_tokens = regex_tokenizer.transform(df_final)

# Eliminar stopwords en español
stopwords = StopWordsRemover.loadDefaultStopWords("spanish")
remover = StopWordsRemover(inputCol="tokens", outputCol="filtered_tokens", stopWords=stopwords)

df_filtered = remover.transform(df_tokens)

In [11]:
df_filtered.select("tokens").show(50)

25/04/09 19:12:58 WARN PythonRunner: Detected deadlock while completing task 1.0 in stage 12 (TID 27): Attempting to kill Python Worker
                                                                                

+--------------------+
|              tokens|
+--------------------+
|[la, montaña, elí...|
|[nota, del, trans...|
|[start, of, the, ...|
|[legends, tales, ...|
|[manfredo, drama,...|
|[nota, del, trans...|
|[la, fontana, de,...|
|[junto, al, pasig...|
|[la, niña, de, lu...|
|[pío, baroja, el,...|
|[nota, del, trans...|
|[nota, del, trans...|
|[nota, del, trans...|
|[la, espuma, obra...|
|[el, paraíso, de,...|
|[logica, de, andr...|
|[online, distribu...|
|[ilustración, jua...|
|[cuentos, de, amo...|
|[marianela, por, ...|
|[bailén, episodio...|
|[dr, jose, rizal,...|
|[illustration, pe...|
|[esta, traducción...|
|[apolinario, mabi...|
|[manuel, romero, ...|
|[viajes, por, fil...|
|[el, gaucho, mart...|
|[diario, de, la, ...|
|[obras, completas...|
|[filosofía, funda...|
|[stan, goodman, d...|
|[derroteros, viag...|
|[trafalgar, benit...|
|[elíseo, reclus, ...|
|[expedicion, de, ...|
|[viajes, por, fil...|
|[viajes, por, fil...|
|[belarmino, apolo...|
|[credits, john, h...|
|[jorge, oh

In [12]:
df_filtered.printSchema()

root
 |-- nombre_archivo: string (nullable = true)
 |-- content: string (nullable = true)
 |-- title: string (nullable = true)
 |-- author: string (nullable = true)
 |-- authoryearofbirth: double (nullable = true)
 |-- authoryearofdeath: double (nullable = true)
 |-- subjects: string (nullable = true)
 |-- tokens: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- filtered_tokens: array (nullable = true)
 |    |-- element: string (containsNull = true)



In [13]:
from pyspark.sql.functions import size

# Agregar una nueva columna 'token_count' con la cantidad de tokens en cada fila
df_filtered = df_filtered.withColumn('token_count', size(df_filtered['tokens']))

In [14]:
# Especificar la ruta local para el archivo Parquet
ruta_local = "libros.parquet"

# Guardar el DataFrame como un archivo Parquet en la ruta especificada, sobrescribiendo si ya existe
df_filtered.write.mode("overwrite").option("maxRecordsPerFile", 1000000).parquet(ruta_local)

In [15]:
from pyspark.sql.functions import sum

# Sumar la columna 'token_count' para obtener la cantidad total de tokens
total_tokens = df_filtered.agg(sum('token_count')).collect()[0][0]

# Mostrar el total de tokens
print("Total de tokens:", total_tokens)

[Stage 28:=============================>                            (1 + 1) / 2]

Total de tokens: 7496845
